# Speech recording with wake word

In order to run this code, essential libraries must be installed first.
- pip3 install pvrecorder
- pip3 install pvporcupine
- pip3 install pyaduio
- pip3 install wave
In addition, attain an access key from picovoice corporation.

## pvporcupine

#### Import related libraries

In [1]:
import argparse
import wave
from pvrecorder import PvRecorder
import pvporcupine

#### Initialization

In [2]:
porcupine_api = pvporcupine.create(access_key='access key attained from picovoice corp', keywords=['jarvis'])

#### pvporcupine function

pvporcupine detects audio keyword

In [3]:
def pvporcupine():
    try:
        recorder = PvRecorder(device_index=1, frame_length=porcupine_api.frame_length)
        recorder.start()
        
        print("[pvporcupine listening for keyword...]")
        
        while True:
            pcm = recorder.read()
            keyword_index = porcupine_api.process(pcm)
            if keyword_index >= 0:
                print("[Keyword detected...]")
                return
    except KeyboardInterrupt:
        print("[pvporcupine stopping...]")
    finally:
        if recorder is not None:
            recorder.delete()
        porcupine_api.delete()

## pyaudio

#### Import realted libraries

In [4]:
import pyaudio
import wave

#### Initialization

In [5]:
audio = pyaudio.PyAudio()

#### Listing potential input device which is connected

In [6]:
for index in range(audio.get_device_count()):
    desc = audio.get_device_info_by_index(index)
    print("{index}: {device}".format(index=index, device=desc["name"]))

0: tegra-hda: HDMI 0 (hw:0,3)
1: tegra-snd-t210ref-mobile-rt565x: - (hw:1,0)
2: tegra-snd-t210ref-mobile-rt565x: - (hw:1,1)
3: tegra-snd-t210ref-mobile-rt565x: - (hw:1,2)
4: tegra-snd-t210ref-mobile-rt565x: - (hw:1,3)
5: tegra-snd-t210ref-mobile-rt565x: - (hw:1,4)
6: tegra-snd-t210ref-mobile-rt565x: - (hw:1,5)
7: tegra-snd-t210ref-mobile-rt565x: - (hw:1,6)
8: tegra-snd-t210ref-mobile-rt565x: - (hw:1,7)
9: tegra-snd-t210ref-mobile-rt565x: - (hw:1,8)
10: tegra-snd-t210ref-mobile-rt565x: - (hw:1,9)
11: USB Audio Device: - (hw:2,0)


#### pyaudio option values

In [7]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
INPUT_DEVICE_INDEX = 11
CHUNK = 4096
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "recording.wav"

#### pyaudio function

In [8]:
def pyaudio():
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, input_device_index=INPUT_DEVICE_INDEX, frames_per_buffer=CHUNK)
    frames = []
    
    print("[pyaudio recording started...]")
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("[pyaudio reocrding finished...]")
    
    stream.stop_stream()
    stream.close()
    audio.terminate()
    
    wf = wave.open(WAVE_OUTPUT_FILENAME, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    return

### Main function

When keyword(in this case "jarvis") is detected from pvporcupine, pyaudio will be start recording the input voice for 5 seconds and save the recorded file to the current repository as "reocrding.wave".

In [9]:
pvporcupine()
pyaudio()

[pvporcupine listening for keyword...]
[Keyword detected...]
[pyaudio recording started...]
[pyaudio reocrding finished...]
